In [8]:
import pandas as pd
import tools as t

Предобработка тектов вакансий

Добавить сюда ID вакансий и key_skills

In [9]:
df_descr = pd.DataFrame(pd.read_json('deduplicated.json')[['id', 'description', 'key_skills']])

Удаление нерелевантных символов

In [10]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

In [11]:
df_descr['d_clean'] = df_descr['description'].apply(lambda x: t.clean_text(x, not_to_split))
df_descr['ks_clean'] = df_descr['key_skills'].apply(lambda x: t.clean_text(str(x), not_to_split))

Токенизация

In [12]:
from nltk import word_tokenize

In [13]:
df_descr['d_tokens'] = df_descr['d_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))
df_descr['ks_tokens'] = df_descr['ks_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))

Лемматизация (23771 текста обрабатывались 55 мин 29 сек)

In [14]:
df_descr['d_lemmatized'] = df_descr['d_tokens'].apply(lambda x: t.lemmatization(x))

In [15]:
df_descr['ks_lemmatized'] = df_descr['ks_tokens'].apply(lambda x: t.lemmatization(x))

In [16]:
df_descr['ks_clean'][df_descr['ks_clean'].str.contains('excel') == True]

3650     ms powerpoint ms excel работа в режиме многоза...
2503     грамотная речь работа в команде креативность п...
3684     возможное обучение повышение квалификации ms w...
3582     анализ рынка маркетинговые исследования swot а...
2228     excel подготовка презентаций сбор и анализ инф...
                               ...                        
86462    ms powerpoint работа с большим объемом информа...
88176    яндекс директ яндекс метрика google analytics ...
88175    яндекс директ яндекс метрика google analytics ...
88177    яндекс директ яндекс метрика google analytics ...
88178    яндекс директ яндекс метрика google analytics ...
Name: ks_clean, Length: 2445, dtype: object

Скачивание стоп-слов для русского языка и добавление новых стоп-слов

In [17]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

Удаление стоп-слов

In [18]:
df_descr['d_wo_sw'] = df_descr['d_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))
df_descr['ks_wo_sw'] = df_descr['ks_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))

In [19]:
df_full = pd.DataFrame(pd.read_json('deduplicated.json'))

In [20]:
df_descr['proffesional_roles'] = df_full['proffesional_roles']

In [21]:
df_descr.to_json('preprocessed_text.json')

In [22]:
# df_descr = pd.read_json('preprocessed_text.json')

In [23]:
df_descr['d_full'] = df_descr['d_wo_sw'].apply(lambda x: ' '.join(x))
df_descr['ks_full'] = df_descr['ks_wo_sw'].apply(lambda x: ' '.join(x))

SKILL COUNT BASED ON DICTIONARY

Создаем датафрейм из id, полного описания вакансии и key_skills, добавляем столбец с пустыми списками (для полученных навыков) 

In [24]:
descr = pd.DataFrame({'id': df_descr['id'], 'd_full': df_descr['d_full'], 'ks_full': df_descr['ks_full']})
descr['extracted_skills'] = descr.apply(lambda _: [], axis = 1)

Создаем словарь навыков

In [25]:
df_dict = pd.read_excel('automatic generated dict.xlsx')['name'].to_list()
skills = dict.fromkeys(df_dict, 0)

Функция для подсчета частоты встечаемости навыков и вытаскивания навыков в отдельное поле

In [26]:
import regex as re

In [27]:
def skills_count(id):
    ind = descr.index[descr['id'] == id].tolist()[0]
    text = descr['d_full'][ind]
    ks = descr['ks_full'][ind]
    for skill in skills:
        if re.search(skill, ks) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1
        elif re.search(skill, text) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1

In [28]:
descr['id'].apply(lambda x: skills_count(x))

3639     None
3640     None
3641     None
3642     None
3643     None
         ... 
87787    None
88178    None
86084    None
86107    None
86109    None
Name: id, Length: 23771, dtype: object

Сортировка

In [29]:
skills = dict(sorted(skills.items(), key=lambda item: item[1], reverse=True))

In [30]:
skills

{'sql': 9605,
 'бизнес процесс': 8271,
 'excel': 7497,
 'erp': 5787,
 '1с': 5733,
 'высокий образование': 5466,
 'технический задание': 5413,
 'моделирование': 5265,
 'bpmn': 5007,
 'аналитический мышление': 4339,
 'анализ данные': 4060,
 'база данные': 4030,
 'uml': 3984,
 'ответственность': 3855,
 'английский язык': 3678,
 'бизнес анализ': 3538,
 'jira': 3111,
 'powerpoint': 3063,
 'python': 2970,
 'rest': 2853,
 'аналитический склад ум': 2805,
 'системный анализ': 2778,
 'больший объём информация': 2754,
 'office': 2695,
 'confluence': 2635,
 'api': 2576,
 'разработка технический задание': 2551,
 'технический документация': 2481,
 'постановка задача разработчик': 2470,
 'проектный документация': 2329,
 'моделирование бизнес процесс': 2325,
 'описание бизнес процесс': 2285,
 'консультирование': 2183,
 'agile': 2091,
 'многозадачность': 1981,
 'внимательность': 1945,
 'soap': 1920,
 'power bi': 1845,
 'статистика': 1819,
 'сводный таблица': 1804,
 'бизнес аналитика': 1798,
 'xml': 176